# Generador de Haikus 3.0
En este Notebook se explicará el tercer acercamiento a la creación de un generador de haikus realizado con Deep Learning. Intentaremos replicar el experimento explicado en el artículo: https://towardsdatascience.com/generating-haiku-with-deep-learning-dbf5d18b4246

Intentaremos lograr la generación de haikus a nivel de carácter haciendo uso de una red neuronal recurrente con una capa LSTM que reciba los tres versos como entrada simultáneamente, así como la longitud de los versos en sílabas.

A continuación podemos ver todos los módulos que serán usados en el presente código.

In [2]:
from pathlib import Path
import re
import pandas as pd
import numpy as np
from keras.callbacks import ModelCheckpoint,  CSVLogger
from keras.layers import Add, Dense, Input, LSTM, Activation
from keras.models import Model, Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from keras.models import Sequential
from keras.optimizers import RMSprop
import inflect
import tensorflow as tf
from keras import backend as K

## 1. Preprocesamiento
En este bloque de código configuramos la ruta al dataset en nuestra máquina.

In [3]:
root_path = Path('')
haiku_path = "all_haiku.csv"

Para este experimento necesitaremos contar el número de sílabas que tiene cada uno de los versos de los poemas. Para ello haremos uso del diccionario **CMU**, que podemos encontrar en http://www.speech.cs.cmu.edu/tools/lextool.html

Este diccionario almacena un conjunto de fonemas que pueden contar como una sílaba para distintas palabras.

También incluimos un fragmento de código para poder añadir palabras que el diccionario no tenga en cuenta, éstas deberán ser introducidas en el archivo "*custom.dict.txt*" siguiendo el formato del diccionario **CMU**.

Para este experimento no haremos uso de ésta funcionalidad, solo haremos uso de las palabras predeterminadas del diccionario.

In [4]:
# Carga los fonemas

# Diccionario estándar
WORDS = {}
with (root_path / 'cmudict.dict.txt').open('r') as f:
    for line in f.readlines():
        word, phonemes = line.strip().split(' ', 1)
        word = re.match(r'([^\(\)]*)(\(\d\))*', word).groups()[0]
        phonemes = phonemes.split(' ')
        syllables = sum([re.match(r'.*\d', p) is not None for p in phonemes])
        #print(word, phonemes, syllables)
        if word not in WORDS:
            WORDS[word] = []
        WORDS[word].append({
            'phonemes': phonemes,
            'syllables': syllables
        })
        
CUSTOM_WORDS = {}
vowels = ['AA', 'AE', 'AH', 'AO', 'AW', 'AX', 'AXR', 'AY', 'EH', 'ER', 'EY', 'IH', 'IX', 'IY', 'OW', 'OY', 'UH', 'UW', 'UX']
with (root_path / 'custom.dict.txt').open('r') as f:
    for line in f.readlines():
        try:
            word, phonemes = line.strip().split('\t', 1)
        except:
            print(line)
            continue
        word = re.match(r'([^\(\)]*)(\(\d\))*', word).groups()[0].lower()
        phonemes = phonemes.split(' ')
        syllables = sum([(p in vowels) for p in phonemes])
        
        if word not in CUSTOM_WORDS:
            CUSTOM_WORDS[word] = []
        CUSTOM_WORDS[word].append({
            'phonemes': phonemes,
            'syllables': syllables
        })

Una vez tenemos cargados los datos del diccionario de pronunciación, definiremos las funciones que leerán las líneas de los poemas, las limpiarán de símbolos y números y posteriormente contarán las sílabas que tienen.

In [5]:
inflect_engine = inflect.engine()

# Diccionario de palabras no encontradas, deberán buscarse los fonemas
NOT_FOUND = set()

def get_words(line):
    
    # Creamos una lista con las palabras del verso
    
    line = str(line)
    line = line.lower()
    # Reemplaza las números con su palabra escrita normalmente
    ws = []
    for word in line.split(' '):
        if re.search(r'\d', word):
            x = inflect_engine.number_to_words(word).replace('-', ' ')
            ws = ws + x.split(' ')
        else:
            ws.append(word)

    line = ' '.join(ws)

    words = []
    for word in line.split(' '):
        word = re.match(r'[\'"]*([\w\']*)[\'"]*(.*)', word).groups()[0]
        word = word.replace('_', '')
        words.append(word)
        
    return words

def count_non_standard_words(line):
    
    # Cuenta el número de palabras en el verso que no aparecen en el diccionario CMU.
    
    count = 0
    for word in get_words(line):
        if word and (word not in WORDS):
            count += 1
    return count

def get_syllable_count(line):
    # Obtiene los posibles números de sílaba de la línea

    counts = [0]
    return_none = False
    for word in get_words(line):
        try:
            if word:
                if (word not in WORDS) and (word not in CUSTOM_WORDS):
                    word = word.strip('\'')
                    
                if word in WORDS:
                    syllables = set(p['syllables'] for p in WORDS[word])
                else:
                    syllables = set(p['syllables'] for p in CUSTOM_WORDS[word])
                #print(syllables)
                new_counts = []
                for c in counts:
                    for s in syllables:
                        new_counts.append(c+s)

                counts = new_counts
        except:
            NOT_FOUND.add(word)
            return_none = True

    if return_none:
        return None
    
    return ','.join([str(i) for i in set(counts)])

Leeremos los poemas del dataset

In [6]:
all_haikus = pd.read_csv(str(haiku_path), usecols=["0","1","2"])

all_haikus

,0,1,2
0,fishing boats,colors of,the rainbow
1,ash wednesday--,trying to remember,my dream
2,snowy morn--,pouring another cup,of black coffee
3,shortest day,flames dance,in the oven
4,haze,half the horse hidden,behind the house
...,...,...,...
144118,I'm not asking did,you say it nor clarify,what you said neither
144119,You are truly a,moron or a liar I'm,inclined to think both
144120,Ain't no selfie on,this earth that's gonna make me,like Theresa May
144121,is doing a great,job turning Independents,into Democrats


Ahora aplicaremos las funciones que definimos anteriormente. Contaremos las sílabas de todos los versos y eliminaremos los poemas que tengan más de 3 palabras desconocidas.

También almacenaremos en "*unrecognized_words.txt*" las palabras que no se han podido reconocer. Para mejorar el funcionamiento de este modelo podrían añadirse éstas palabras al diccionario personalizable.

In [7]:
# Borra haikus con más de 3 palabras desconocidas
all_haikus['unknown_word_count'] = np.sum([all_haikus[str(i)].apply(count_non_standard_words) for i in range(3)], axis=0)
all_haikus = all_haikus[all_haikus['unknown_word_count'] < 3].copy()

for i in range(3):
    all_haikus['%s_syllables' % i] = all_haikus[str(i)].apply(get_syllable_count)
    
print("Palabras no reconocidas: ", len(NOT_FOUND))

with open('unrecognized_words.txt', 'w') as f:
    for w in NOT_FOUND:
        f.write(w)
        f.write('\n')

all_haikus

Palabras no reconocidas:  5796


,0,1,2,unknown_word_count,0_syllables,1_syllables,2_syllables
0,fishing boats,colors of,the rainbow,0,3,3,3
1,ash wednesday--,trying to remember,my dream,0,3,"5,6",2
2,snowy morn--,pouring another cup,of black coffee,0,3,6,4
3,shortest day,flames dance,in the oven,0,3,2,4
4,haze,half the horse hidden,behind the house,0,1,5,4
...,...,...,...,...,...,...,...
144118,I'm not asking did,you say it nor clarify,what you said neither,0,5,7,5
144119,You are truly a,moron or a liar I'm,inclined to think both,0,5,7,5
144120,Ain't no selfie on,this earth that's gonna make me,like Theresa May,0,5,7,5
144121,is doing a great,job turning Independents,into Democrats,0,5,7,5


Ahora haremos algo que no se lleva a cabo en el artículo. Debido a que nosotros tenemos un conjunto de haikus lo suficientemente grande, nos tomaremos la licencia de quedarnos solamente con los que tengan estructura 5-7-5.

In [8]:
# Filtramos el conjunto de poemas y nos quedamos con los que tengan estructura 5-7-5
corpus = []
all_haikus = all_haikus[all_haikus['0_syllables']=="5"]
all_haikus = all_haikus[all_haikus['1_syllables']=="7"]
all_haikus = all_haikus[all_haikus['2_syllables']=="5"]
df = all_haikus

In [9]:
inputs = df[['0', '1', '2']].values
inputs[:, 0]

array(['visiting the graves', 'profound blue of night ',
       'scattered in the ditch ', ..., "Ain't no selfie on",
       'is doing a great', 'Wanted to send a'], dtype=object)

Comenzamos a preparar los datos para el entrenamiento. Los inputs repetirán el primer carácter y los outputs se completarán añadiendo el próximo carácter, a excepción del último verso, que no lo hará.

In [32]:
# Tomamos el máximo tamaño de línea de todos los versos y eliminamos las muestras
# que sean más largas que el 99 percentil de longitud.

max_line_length = int(max([df['%s' % i].str.len().quantile(.99) for i in range(3)]))
df = df[
    (df['0'].str.len() <= max_line_length) & 
    (df['1'].str.len() <= max_line_length) & 
    (df['2'].str.len() <= max_line_length)
].copy()
df
# Aplicamos padding a los versos que no tienen el máximo tamaño de línea
# añadiendo \n al final de los mismos hasta llegar a este tamaño máximo
for i in range(3):
    # Para los inputs, duplicaremos el primer caracter
    df['%s_in' % i] = (df[str(i)].str[0] + df[str(i)]).str.pad(max_line_length+2, 'right', '\n')
    
    # Añadimos el primer carácter de la próxima línea si no es el último verso
    if i == 2: # Si es el último verso
        df['%s_out' % i] = df[str(i)].str.pad(max_line_length+2, 'right', '\n')
    else: # Si no es el último verso se añade el primer carácter de la próxima línea
        # Esto ayudará con el entrenamiento, de manera que la siguiente RNR tenga mejores probabilidades de
        # tomar el primer carácter correctamente.
        df['%s_out' % i] = (df[str(i)] + '\n' + df[str(i+1)].str[0]).str.pad(max_line_length+2, 'right', '\n')
    
max_line_length += 2

df

,0,1,2,unknown_word_count,0_syllables,1_syllables,2_syllables,0_in,0_out,1_in,1_out,2_in,2_out
24,visiting the graves,stronger the October wind,at my grandparents',0,5,7,5,vvisiting the graves\n\n\n\n\n\n\n\n\n\n\n\n\n...,visiting the graves\ns\n\n\n\n\n\n\n\n\n\n\n\n...,sstronger the October wind\n\n\n\n\n\n\n\n\n\n,stronger the October wind\na\n\n\n\n\n\n\n\n\n,aat my grandparents'\n\n\n\n\n\n\n\n\n\n\n\n\n...,at my grandparents'\n\n\n\n\n\n\n\n\n\n\n\n\n\...
141,profound blue of night,the resin and salt of pines,so far from the sea,0,5,7,5,pprofound blue of night \n\n\n\n\n\n\n\n\n\n\n\n,profound blue of night \nt\n\n\n\n\n\n\n\n\n\n\n,tthe resin and salt of pines\n\n\n\n\n\n\n\n,the resin and salt of pines\ns\n\n\n\n\n\n\n,sso far from the sea\n\n\n\n\n\n\n\n\n\n\n\n\n...,so far from the sea\n\n\n\n\n\n\n\n\n\n\n\n\n\...
142,scattered in the ditch,like tiny scraps of blue sky,bits of plastic bag,0,5,7,5,sscattered in the ditch \n\n\n\n\n\n\n\n\n\n\n\n,scattered in the ditch \nl\n\n\n\n\n\n\n\n\n\n\n,llike tiny scraps of blue sky\n\n\n\n\n\n\n,like tiny scraps of blue sky\nb\n\n\n\n\n\n,bbits of plastic bag\n\n\n\n\n\n\n\n\n\n\n\n\n...,bits of plastic bag\n\n\n\n\n\n\n\n\n\n\n\n\n\...
343,the smell of her hands,on the neck of the bottle,drinking greedily,0,5,7,5,tthe smell of her hands\n\n\n\n\n\n\n\n\n\n\n\n\n,the smell of her hands\no\n\n\n\n\n\n\n\n\n\n\n\n,oon the neck of the bottle\n\n\n\n\n\n\n\n\n\n,on the neck of the bottle\nd\n\n\n\n\n\n\n\n\n,ddrinking greedily\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,drinking greedily\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
435,christmas services,a cellular phone rings out,handel's messiah,0,5,7,5,cchristmas services\n\n\n\n\n\n\n\n\n\n\n\n\n\...,christmas services\na\n\n\n\n\n\n\n\n\n\n\n\n\...,aa cellular phone rings out\n\n\n\n\n\n\n\n\n,a cellular phone rings out\nh\n\n\n\n\n\n\n\n,hhandel's messiah\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,handel's messiah\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
144118,I'm not asking did,you say it nor clarify,what you said neither,0,5,7,5,II'm not asking did\n\n\n\n\n\n\n\n\n\n\n\n\n\...,I'm not asking did\n \n\n\n\n\n\n\n\n\n\n\n\n\...,you say it nor clarify\n\n\n\n\n\n\n\n\n\n\n\n,you say it nor clarify\nw\n\n\n\n\n\n\n\n\n\n\n,wwhat you said neither\n\n\n\n\n\n\n\n\n\n\n\n...,what you said neither\n\n\n\n\n\n\n\n\n\n\n\n\...
144119,You are truly a,moron or a liar I'm,inclined to think both,0,5,7,5,YYou are truly a\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,You are truly a\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n...,moron or a liar I'm\n\n\n\n\n\n\n\n\n\n\n\n\...,moron or a liar I'm\ni\n\n\n\n\n\n\n\n\n\n\n\...,iinclined to think both\n\n\n\n\n\n\n\n\n\n\n\n\n,inclined to think both\n\n\n\n\n\n\n\n\n\n\n\n...
144120,Ain't no selfie on,this earth that's gonna make me,like Theresa May,0,5,7,5,AAin't no selfie on\n\n\n\n\n\n\n\n\n\n\n\n\n\...,Ain't no selfie on\n \n\n\n\n\n\n\n\n\n\n\n\n\...,this earth that's gonna make me\n\n\n,this earth that's gonna make me\nl\n\n,llike Theresa May\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,like Theresa May\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
144121,is doing a great,job turning Independents,into Democrats,0,5,7,5,iis doing a great\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,is doing a great\n \n\n\n\n\n\n\n\n\n\n\n\n\n\...,job turning Independents\n\n\n\n\n\n\n\n\n\n,job turning Independents\ni\n\n\n\n\n\n\n\n\n,iinto Democrats\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,into Democrats\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


Ahora codificaremos los datos haciendo uso de codificación **one-hot** para que puedan ser utilizados por el modelo.

In [33]:
inputs = df[['0_in', '1_in', '2_in']].values

tokenizer = Tokenizer(filters='', char_level=True)
tokenizer.fit_on_texts(inputs.flatten())
n_tokens = len(tokenizer.word_counts) + 1
aux_X = [tokenizer.texts_to_sequences(inputs[:,i]) for i in range(3)]

# X es la entrada de cada línea en secuencias de caracteres codificados en one-hot
X = np_utils.to_categorical(np.array(aux_X), num_classes=n_tokens)

outputs = df[['0_out', '1_out', '2_out']].values

# Y es la entrada de cada línea en secuencias de caracteres codificados en one-hot
Y = np_utils.to_categorical([tokenizer.texts_to_sequences(outputs[:,i]) for i in range(3)], num_classes=n_tokens)

# X_syllables es la cuenta de sílabas de cada línea
X_syllables = df[['0_syllables', '1_syllables', '2_syllables']].values

## 2. Creación del modelo
Para este modelo, necesitaremos utilizar clases especiales haciendo uso de la API funcional de Keras. Estas clases constituirán las distintas líneas de entrenamiento que se usarán en la capa LSTM, de manera que puedan entrenarse simultáneamente.

Crearemos una clase **TrainingLine** que constituirá una de las líneas de entrenamiento. Después definiremos una función *create_training_model* que creará las líneas de entrenamiento en función de los parámetros que consideremos.

In [34]:
class TrainingLine:
    def __init__(self, name, previous_line, lstm, n_tokens):
        self.char_input = Input(shape=(None, n_tokens), name='char_input_%s' % name)

        self.syllable_input = Input(shape=(1,), name='syllable_input_%s' % name)
        self.syllable_dense = Dense(lstm.units, activation='relu', name='syllable_dense_%s' % name)
        self.syllable_dense_output = self.syllable_dense(self.syllable_input)

        #self.lstm = LSTM(latent_dim, return_state=True, return_sequences=True, name='lstm_%s' % name)

        #Si hay una linea previa, el estado inicial de ésta nueva será el que de como salida la anterior
        if previous_line:
            initial_state = [
                Add(name='add_h_%s' % name)([
                    previous_line.lstm_h,
                    self.syllable_dense_output
                ]),
                Add(name='add_c_%s' % name)([
                    previous_line.lstm_c,
                    self.syllable_dense_output
                ])
            ]
        else:
            initial_state = [self.syllable_dense_output, self.syllable_dense_output]

        self.lstm_out, self.lstm_h, self.lstm_c = lstm(self.char_input, initial_state=initial_state)

        self.output_dense = Dense(n_tokens, activation='softmax', name='output_%s' % name)
        self.output = self.output_dense(self.lstm_out)

def create_training_model(latent_dim, n_tokens):
    lstm = LSTM(latent_dim, return_state=True, return_sequences=True, name='lstm')
    lines = []
    inputs = []
    outputs = []
#Se crean todas las lineas de inputs, que se anclan a la capa LSTM que creamos previamente
    for i in range(3):
        previous_line = lines[-1] if lines else None
        lines.append(TrainingLine('line_%s' % i, previous_line, lstm, n_tokens))
        inputs += [lines[-1].char_input, lines[-1].syllable_input]
        outputs.append(lines[-1].output)

    training_model = Model(inputs, outputs)
    training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

    return training_model, lstm, lines, inputs, outputs

## 3. Entrenamiento del modelo
A continuación definiremos y entrenaremos el modelo haciendo uso de las clases definidas en el apartado anterior.

La red consistirá en una capa LSTM de 2048 neuronas a la que se le pasarán 3 líneas de entrenamiento. En cada una de estas líneas se pasará un verso y las sílabas que tiene como estado. El número de sílabas, además, pasará por una capa Dense de tantas neuronas como tendrá la capa LSTM, con activación ***relu***. Por último, cada línea terminará con una capa Dense con tantas neuronas como caracteres se estén utilizando y función de activación ***softmax***.

Entrenaremos la red durante 10 épocas con lotes de 256 muestras y reservando un 10% de las muestras para la validación. Monitorizaremos los resultados de cada época teniendo en cuenta la función de pérdida "***categorical_crossentropy***" y la precisión. Además, conforme pasen las épocas almacenaremos los pesos que mejores resultados hayan dado respecto a la pérdida.

In [36]:
tf_session = tf.Session()
K.set_session(tf_session)


output_dir = Path('output_%s' % "salida")
epochs = 10
X_syllables = np.array(X_syllables)
X_syllables = X_syllables.astype(np.float)
training_model, lstm, lines, inputs, outputs = create_training_model(2048, n_tokens)

filepath = str(output_dir / ("%s-{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5" % 2048))
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

csv_logger = CSVLogger(str(output_dir / 'training_log.csv'), append=True, separator=',')

callbacks_list = [checkpoint, csv_logger]

history = training_model.fit([
    X[0], X_syllables[:,0], 
    X[1], X_syllables[:,1], 
    X[2], X_syllables[:,2]
], [Y[0], Y[1], Y[2]], batch_size=256, epochs=epochs, validation_split=.1, callbacks=callbacks_list)

Train on 82574 samples, validate on 9175 samples
Epoch 1/10
82574/82574 [==============================] - 330s 4ms/step - loss: 5.6484 - output_line_0_loss: 1.5993 - output_line_1_loss: 2.3260 - output_line_2_loss: 1.7208 - output_line_0_acc: 0.5791 - output_line_1_acc: 0.3591 - output_line_2_acc: 0.5600 - val_loss: 3.8988 - val_output_line_0_loss: 1.0358 - val_output_line_1_loss: 1.7184 - val_output_line_2_loss: 1.1443 - val_output_line_0_acc: 0.6972 - val_output_line_1_acc: 0.4861 - val_output_line_2_acc: 0.6662

Epoch 00001: loss improved from inf to 5.64845, saving model to output_salida\2048-01-5.65-3.90.hdf5
Epoch 2/10
82574/82574 [==============================] - 314s 4ms/step - loss: 3.2841 - output_line_0_loss: 0.8550 - output_line_1_loss: 1.4744 - output_line_2_loss: 0.9541 - output_line_0_acc: 0.7432 - output_line_1_acc: 0.5519 - output_line_2_acc: 0.7114 - val_loss: 2.7855 - val_output_line_0_loss: 0.7119 - val_output_line_1_loss: 1.2789 - val_output_line_2_loss: 0.7946 -

Como podemos observar, los resultados son bastante buenos, con una precisión en la validación de alrededor del 80% en el primer verso y en el último.

Sin embargo, el segundo verso obtiene alrededor de un 70% de precisión en la validación, algo que resulta llamativo.

## 5. Evaluación

En esta ocasión trataremos de generar poemas con distintas temperaturas y observaremos la estructura que se construye en los mismos.

Para esto, crearemos la función *sample*, que se encargará de predecir el próximo carácter en base a unas probabilidades y un valor de temperatura. 

También, en este caso, es necesario definir una clase ***GeneratorLine*** que represente las líneas de generador que se asocian a cada una de las ***TrainingLine*** del modelo.

Crearemos una clase ***Generator*** que será la que utilicemos para definir nuestro generador. Esta clase tendrá un método ***generate_haiku*** que recibirá como parámetros un array con las sílabas que quisiéramos que tuviese el haiku, la temperatura a utilizar y el primer carácter utilizado.

Para generar los haikus se ejecutarán algunos métodos en la sesión de TensorFlow tales como ***feed_dict*** para poder introducir los datos según se generen al modelo.

In [48]:
def sample(preds, temperature=0.5):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

class GeneratorLine:
    def __init__(self, name, training_line, lstm, n_tokens):
        self.char_input = Input(shape=(None, n_tokens), name='char_input_%s' % name)

        self.syllable_input = Input(shape=(1,), name='syllable_input_%s' % name)
        self.syllable_dense = Dense(lstm.units, activation='relu', name='syllable_dense_%s' % name)
        self.syllable_dense_output = self.syllable_dense(self.syllable_input)

        self.h_input = Input(shape=(lstm.units,), name='h_input_%s' % name)
        self.c_input = Input(shape=(lstm.units,), name='c_input_%s' % name)
        initial_state = [self.h_input, self.c_input]

        self.lstm = lstm

        self.lstm_out, self.lstm_h, self.lstm_c = self.lstm(self.char_input, initial_state=initial_state)

        self.output_dense = Dense(n_tokens, activation='softmax', name='output_%s' % name)
        self.output = self.output_dense(self.lstm_out)

        self.syllable_dense.set_weights(training_line.syllable_dense.get_weights())
        #self.lstm.set_weights(lstm.get_weights())
        self.output_dense.set_weights(training_line.output_dense.get_weights())

class Generator:
    def __init__(self, lstm, lines, tf_session, tokenizer, n_tokens, max_line_length):
        self.tf_session = tf_session
        self.tokenizer = tokenizer
        self.n_tokens = n_tokens
        self.max_line_length = max_line_length

        self.lstm = LSTM(
            lstm.units, return_state=True, return_sequences=True,
            name='generator_lstm'
        )
        self.lines = [
            GeneratorLine(
                'generator_line_%s' % i,
                lines[i], self.lstm, self.n_tokens
            ) for i in range(3)
        ]
        self.lstm.set_weights(lstm.get_weights())

    def generate_haiku(self, syllables=[5, 7, 5], temperature=.1, first_char=None):
        output = []
        h = None
        c = None

        if first_char is None:
            first_char = chr(int(np.random.randint(ord('a'), ord('z')+1)))

        next_char = self.tokenizer.texts_to_sequences(first_char)[0][0]

        for i in range(3):
            line = self.lines[i]
            
            s = self.tf_session.run(
                line.syllable_dense_output,
                feed_dict={
                    line.syllable_input: [[syllables[i]]]
                }
            )
            if h is None:
                h = s
                c = s
            else:
                h = h + s
                c = c + s

            line_output = [next_char]

            end = False
            next_char = None
            for i in range(self.max_line_length):
                char, h, c = self.tf_session.run(
                    [line.output, line.lstm_h, line.lstm_c],
                    feed_dict={
                        line.char_input: [[
                            np_utils.to_categorical(
                                line_output[-1],
                                num_classes=self.n_tokens
                            )
                        ]],
                        line.h_input: h,
                        line.c_input: c
                    }
                )
                char = sample(char[0,0], temperature)
                if char == 1 and not end:
                    end = True
                if char != 1 and end:
                    next_char = char
                    char = 1

                line_output.append(char)

            cleaned_text = self.tokenizer.sequences_to_texts([
                line_output
            ])[0].strip()[1:].replace(
                '   ', '\n'
            ).replace(' ', '').replace('\n', ' ')
            #print(line_output)
            print(cleaned_text)
            output.append(cleaned_text)

        return output

Creamos un placeholder del modelo y cargamos los persos que deseemos.

In [41]:
# Creamos un nuevo placeholder para el modelo
training_model, lstm, lines, inputs, outputs = create_training_model(2048, n_tokens)

# Cargamos los pesos que queramos, especificando el archivo que los guardó
training_model.load_weights(output_dir / '2048-10-1.51-2.20.hdf5')

Creamos el generador con el modelo instanciado anteriormente.

In [50]:
generator = Generator(lstm, lines, tf_session, tokenizer, n_tokens, max_line_length)

A continuación podemos ver qué es lo que se genera al hacer uso del método ***generate_haiku***, mostrando los valores numéricos que se obtienen antes de ser traducidos a palabras y filtrados para que sean legibles.

In [51]:
generator.generate_haiku(temperature = 0.1)
print()

[16, 16, 8, 6, 4, 3, 13, 2, 4, 10, 3, 2, 18, 6, 8, 13, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
united the wind
[2, 2, 5, 20, 2, 4, 10, 3, 2, 13, 3, 24, 6, 12, 2, 22, 16, 4, 2, 6, 2, 13, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
f the devil but i do
[8, 8, 5, 4, 2, 10, 7, 24, 3, 2, 4, 10, 3, 2, 4, 6, 14, 3, 2, 4, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
not have the time to



Generaremos 50 haikus con distintas temperaturas, siempre con el valor por defecto de sílabas 5-7-5. Confiamos en que esta estructura se mantendrá.

En primer lugar se generarán 50 haikus con temperatura 0.1, después se generarán con temperatura 0.3 y, por último, con 1.

In [27]:
for i in range(50):
    generator.generate_haiku()
    print()

finally got a
tory for the first time i
got to stay home soon

only the stream is
he most beautiful person
in the world to me

unless you have to
e the best thing ever i
wish you could be fine

keep your start because
ou can't have to be the same
as it is a word

very sad sometimes
 stay in bed and watch the
shit out of my head

the more i listen
o the world and then i get
to keep my second

question of the day
 have to get the best of
my life and the rest

question of the day
he strength is the most stressful
people in the world

i wanna go to
he gym but i don't want to
be a good sense of

when you realize
he last time you want to be
right now i love you

kinda wanna go
o the gym but i love the
stream i love this shit

finally got a
ig bank on my face and i
can't stand the shit out

when you realize
he last time i lost you and
you can't have to see

finally got a
tory for the first time i
got to see my screams

u a woman with
he super bowl i would have
to see you good love

have you e

In [23]:
for i in range(50):
    generator.generate_haiku(temperature=.3)
    print()

alright the screams i
on't have to do the same thing
about my day off

people really tell
e why they are not the same
as it is to be

my mom is going
o be a good mom to see
the next week of god

x can't believe i
ot to see the first time i
got to see my screams

keep your mind tonight
hy do you see an instant
sound like the last time

me i'm so beauty
or you now i can't wait to
get a good combin

x moment when you
tart to find something that you
can do to yourself

good morning to all
he boy start and change your life
and it's the truth too

life is a good mome
the story is the lessons
just to stay somewhere

x may the person
ho think they are not the same
as you can do things

change your moms another
or you gotta stop the best
personalities

because the rest of
ou want to be strong enough
to be relevant

how does anyone
hink they are not the same as
the people i love

gotta get my nails
o i can be so into
the shower again

question of the day
 have to be on the street
and start to ge

In [26]:
for i in range(50):
    generator.generate_haiku(temperature=1)
    print()

question through this purk
s cool are the mean says d
everything for you

mom birthday is much
orse she might be fuck bitches
all over igade

virgil in overtime
e is a roair she's me
ryan can't sell out

things a jecky need
ungle dness is beautiful
how long on the heart

kenyrick should be
orn homoreal corolat
heartbreaking and clear

you'll never know who
othing you don't suppose to
be eepent for you

you gotta get up
our club with the right pictures
i just want it nace

forgot to remain
ife in the show guard angry
sent the biation

unlows in missing
s an amazing show i
need to update it

don't look collactic
nion email but for your
movies is sy drunk

like a ticket ass
hotographer i feel like
i'm in front of them

leirn killing land with
hat boam ast house needs to be
greater than the legs

zulled my bi boyhigh
ut have four legend when we
even bout to read

people really just
lear something once somebody
sneeze me up lol

going my hair done
 cant love my people are
tea sisterg is not

Como podemos observar, sorprendentemente la mayoría de poemas mantienen el esquema 5-7-5, probablemente gracias a que se filtraron las muestras para que tuvieran esta estructura.

Observamos que no se generan correctamente las primeras letras de la segunda línea en un gran número de ocasiones, pero esto no es un problema demasiado grave, ya que pueden intuirse con facilidad.

Conforme aumenta la temperatura las palabras se vuelven más aleatorias e impredecibles, creando palabras que no existen en realidad pero que suenan creíbles. También, cuanto más se aumenta este valor menos sentido parecen tener los haikus.

Podemos ver que la estructura, en su mayoría, sintácticamente tiene sentido. Sin embargo vemos que en muchas ocasiones los poemas no llegan a tener demasiado sentido, pero sí que podemos encontrar alguno que sorprende y parecería estar escrito por un humano.

#### Haiku destacado 1

question of the day

I have to get the best of

my life and the rest

#### Haiku destacado 2

just say something that

can happen to me I love

you and my brother

#### Haiku destacado 3

being an adult

is so annoying and I

have no idea